In [1]:
from tensorflow.keras.applications import ResNet101
model1 = ResNet101(include_top=False, weights=None, input_shape=(224,224, 3))
model1.trainable = False

In [2]:
model1.summary()

Model: "resnet101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [3]:
import tensorflow.keras,os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout, ZeroPadding2D, GlobalAveragePooling2D, DepthwiseConv2D
from tensorflow.keras import optimizers
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [4]:
model2 = Sequential([model1])
model2.add(Flatten())
model2.add(Dense(units=1024,activation="relu"))
model2.add(Dense(units=1024,activation="relu"))
model2.add(Dense(units=1, activation="sigmoid"))
model2.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model2.trainable = False
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Functional)       (None, 7, 7, 2048)        42658176  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              102761472 
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1025      
Total params: 146,470,273
Trainable params: 0
Non-trainable params: 146,470,273
_________________________________________________________________


In [6]:
###########################################################
# 데이터 전처리
###########################################################


train_datagen = ImageDataGenerator(rescale=1./255)  # 모든 이미지를 1/255로 스케일을 조정합니다
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "./dataset/train2"
validation_dir = './dataset/vali2'

batch_size = 16
# batch_size = 64


train_generator = train_datagen.flow_from_directory( \
        train_dir,                  # 타깃 디렉터리
        target_size=(224, 224),     # 모든 이미지를 224 × 224 크기
        batch_size=batch_size,
        class_mode='binary')        #

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

Found 802 images belonging to 2 classes.
Found 286 images belonging to 2 classes.


In [7]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)  #(20, 150, 150, 3)
    print('배치 레이블 크기:', labels_batch.shape) #(20,)
    break

배치 데이터 크기: (16, 224, 224, 3)
배치 레이블 크기: (16,)


In [10]:
steps_per_epochs = 50 # 학습데이터의 총 개수 / 배치 사이즈 (소수점 올림)
epochs = 15

history = model2.fit(
      train_generator ,
      steps_per_epoch=steps_per_epochs  ,
      epochs=epochs ,
      validation_data=validation_generator,
      validation_steps=18)

Epoch 1/15
50/50 [==============================] - 19s 383ms/step - loss: 0.2580 - accuracy: 0.8880 - val_loss: 1.0791 - val_accuracy: 0.5769
Epoch 2/15
50/50 [==============================] - 20s 395ms/step - loss: 0.2900 - accuracy: 0.8626 - val_loss: 1.4744 - val_accuracy: 0.5000
Epoch 3/15
50/50 [==============================] - 20s 407ms/step - loss: 0.2086 - accuracy: 0.9186 - val_loss: 1.1977 - val_accuracy: 0.5629
Epoch 4/15
50/50 [==============================] - 20s 402ms/step - loss: 0.2497 - accuracy: 0.8944 - val_loss: 2.3086 - val_accuracy: 0.5000
Epoch 5/15
50/50 [==============================] - 20s 406ms/step - loss: 0.1710 - accuracy: 0.9402 - val_loss: 1.6497 - val_accuracy: 0.5000
Epoch 6/15
50/50 [==============================] - 20s 409ms/step - loss: 0.1858 - accuracy: 0.9300 - val_loss: 1.5508 - val_accuracy: 0.5175
Epoch 7/15
50/50 [==============================] - 20s 404ms/step - loss: 0.1417 - accuracy: 0.9529 - val_loss: 1.2425 - val_accuracy: 0.4930

In [11]:
model2.save("./tn_model/res.h5")

D:\Program Files\python38\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
